# Starbucks Stores Analysis

In [22]:
# Housekeeping
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Datasets

Data Constraints:
- Both Starbucks and US datasets published in 2017.
- Starbucks store locations limited to US country. 
- Starbucks store limited to Starbucks brand (no Teavana)
- Exclude Puerto Rico from US datasets

In [23]:
starbucks = pd.read_csv('data/directory.csv')
starbucks = starbucks.query("Brand == 'Starbucks'").query("Country == 'US'")
starbucks = starbucks.drop(columns=["Brand", "Store Name", "Ownership Type", "Street Address","Phone Number","Timezone", "Postcode", "Country"])
starbucks = starbucks.rename(columns={'State/Province' : 'State'})

In [24]:
cities = pd.read_csv('data/uscities.csv')
cities = cities[["city", "state_id", "state_name", "county_name"]]

In [25]:
demographic = pd.read_csv('data/demo.csv', encoding='cp1252')
demographic = demographic[demographic['State'] != 'Puerto Rico']
demographic["County"] = demographic["County"].apply(lambda x: ' '.join(x.split()[0:-1]))

### Merge Data 

In [26]:
mapping = pd.merge(starbucks, cities, left_on=["City", "State"], right_on=["city", "state_id"])
mapping = mapping.drop(columns=["state_id", "city", "State"])
mapping = mapping.rename(columns={"state_name":"State", "county_name":"County"})
mapping

,Store Number,City,Longitude,Latitude,State,County
0,3513-125945,Anchorage,-149.78,61.21,Alaska,Anchorage
1,74352-84449,Anchorage,-149.84,61.14,Alaska,Anchorage
2,12449-152385,Anchorage,-149.85,61.11,Alaska,Anchorage
3,24936-233524,Anchorage,-149.89,61.13,Alaska,Anchorage
4,8973-85630,Anchorage,-149.86,61.14,Alaska,Anchorage
...,...,...,...,...,...,...
12119,22353-220004,Lander,-108.75,42.84,Wyoming,Fremont
12120,74385-87621,Laramie,-105.59,41.32,Wyoming,Albany
12121,73320-24375,Laramie,-105.56,41.31,Wyoming,Albany
12122,22425-219024,Laramie,-105.56,41.31,Wyoming,Albany


***



## Data Analysis

In [28]:
storecount = mapping.groupby(['County', 'State'])['Store Number'].count().to_frame().reset_index()
storecount = storecount.rename(columns={"Store Number":"Count"})
storecount

,County,State,Count
0,Ada,Idaho,32
1,Adair,Missouri,1
2,Adams,Colorado,62
3,Adams,Illinois,2
4,Adams,Pennsylvania,1
...,...,...,...
1026,York,Pennsylvania,10
1027,York,South Carolina,8
1028,York,Virginia,2
1029,Yuba,California,1


In [29]:
df = storecount.merge(demographic, how='right', left_on=['County', 'State'], right_on=['County', 'State']).drop(columns=["Unnamed: 0", "CountyId", "VotingAgeCitizen"])
df['Count'] = df['Count'].fillna(0)
df['Men'] = (df['Men']/df['TotalPop'])*100
df['Women'] = (df['Women']/df['TotalPop'])*100
df

,County,State,Count,TotalPop,Men,Women,Hispanic,White,Black,Native,...,Walk,OtherTransp,WorkAtHome,MeanCommute,Employed,PrivateWork,PublicWork,SelfEmployed,FamilyWork,Unemployment
0,Autauga,Alabama,2.0,55036,48.875282,51.124718,2.7,75.4,18.9,0.3,...,0.6,1.3,2.5,25.8,24112,74.1,20.2,5.6,0.1,5.2
1,Baldwin,Alabama,5.0,203360,48.941286,51.058714,4.4,83.1,9.5,0.8,...,0.8,1.1,5.6,27.0,89527,80.7,12.9,6.3,0.1,5.5
2,Barbour,Alabama,0.0,26201,53.341476,46.658524,4.2,45.7,47.8,0.2,...,2.2,1.7,1.3,23.4,8878,74.1,19.1,6.5,0.3,12.4
3,Bibb,Alabama,0.0,22580,54.255979,45.744021,2.4,74.6,22.0,0.4,...,0.3,1.7,1.5,30.0,8171,76.0,17.4,6.3,0.3,8.2
4,Blount,Alabama,0.0,57667,49.404339,50.595661,9.0,87.4,1.5,0.3,...,0.4,0.4,2.1,35.0,21380,83.9,11.9,4.0,0.1,4.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3137,Sweetwater,Wyoming,1.0,44527,51.611382,48.388618,16.0,79.6,0.8,0.6,...,2.8,1.3,1.5,20.5,22739,78.4,17.8,3.8,0.0,5.2
3138,Teton,Wyoming,2.0,22923,53.086420,46.913580,15.0,81.5,0.5,0.3,...,11.7,3.8,5.7,14.3,14492,82.1,11.4,6.5,0.0,1.3
3139,Uinta,Wyoming,0.0,20758,51.030928,48.969072,9.1,87.7,0.1,0.9,...,1.1,1.3,2.0,19.9,9528,71.5,21.5,6.6,0.4,6.4
3140,Washakie,Wyoming,0.0,8253,49.897007,50.102993,14.2,82.2,0.3,0.4,...,6.9,1.3,4.4,14.3,3833,69.8,22.0,8.1,0.2,6.1


### Correlation

In [30]:
var = ['Count', 'TotalPop', 'Men', 'Women', 'Hispanic', 'White', 'Black', 'Native', 'Asian', 'Pacific',\
       'Income', 'IncomeErr', 'IncomePerCap', 'IncomePerCapErr', 'Poverty', 'ChildPoverty', \
       'Professional', 'Service', 'Office', 'Construction', 'Production', 'Drive', 'Carpool', 'Transit', 'Walk', \
       'OtherTransp', 'WorkAtHome', 'MeanCommute', 'Employed','PrivateWork', 'PublicWork', 'SelfEmployed', \
       'FamilyWork', 'Unemployment']
corr = df.query("Count > 0")[var].corr().drop('Count')[['Count']]
corr

,Count
TotalPop,0.898447
Men,-0.052407
Women,0.052407
Hispanic,0.245240
White,-0.326448
Black,0.072888
Native,-0.045508
Asian,0.446331
Pacific,0.073763
Income,0.209247


***

## Data Visualization